In [1]:
function createGraphFromFile(filename)
	f = open(filename)
	lines = readlines(f)
	firstLine = split(lines[1])
	n = parse(Int64, firstLine[1]) #número de vértices
	g = parse(Int64, firstLine[2]) #número de grupos
	secondLine = split(lines[2])
    L = zeros(g) #limite inferior do grupo j
    U = zeros(g) #limite superior do grupo j
    for j=1:g
    	L[j] = parse(Float64, secondLine[j*2-1])
    	U[j] = parse(Float64, secondLine[j*2])
    end
    p = zeros(n) #peso do vértice i
    thirdLine = split(lines[3])
    for i=1:n
    	p[i] = parse(Float64, thirdLine[i])
    end
    numberOfLines = size(lines)[1]
    A = zeros(n,n) #matriz de adjacência em que cada elemento representa o peso da aresta entre o vértice u e v
    for lineIndex=4:numberOfLines
    	currentLine = split(lines[lineIndex])
    	u = parse(Int64, currentLine[1]) + 1
        v = parse(Int64, currentLine[2]) + 1
        A[u,v] = parse(Float64, currentLine[3])
    end
    return n, g, p, L, U, A
end

n, g, p, L, U, A = createGraphFromFile("D:/UFRGS/Quinto Semestre/Otimização Combinatória/Trabalho final/gbmv240_04.ins")

(240, 12, [2.0, 7.0, 10.0, 2.0, 2.0, 10.0, 10.0, 2.0, 5.0, 4.0  …  2.0, 7.0, 6.0, 1.0, 5.0, 7.0, 6.0, 2.0, 10.0, 10.0], [75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0], [125.0, 125.0, 125.0, 125.0, 125.0, 125.0, 125.0, 125.0, 125.0, 125.0, 125.0, 125.0], [0.0 10.583 … 66.477 75.48; 0.0 0.0 … 20.727 53.472; … ; 0.0 0.0 … 0.0 19.611; 0.0 0.0 … 0.0 0.0])

In [2]:
using JuMP
using GLPKMathProgInterface
m = Model(solver=GLPKSolverMIP(msg_lev=GLPK.MSG_ALL, tm_lim=3600000, out_frq=1))

@variable(m, x[1:n, 1:g], Bin) #g é o número de grupos
@variable(m, c[1:n, 1:n, 1:g], Bin) #g é o número de grupos
@objective( m, Max, sum( c[i,k,j]*A[i,k] for i in 1:n for k in 1:n for j in 1:g ) )
@constraints(m, begin
[i in 1:n], sum(x[i,j] for j in 1:g)==1
[j in 1:g], L[j]<=sum( x[i,j]*p[i] for i in 1:n )<=U[j]
[i in 1:n, k in 1:n, j in 1:g], c[i,k,j]<=(x[i,j]+x[k,j])/2 # isso daqui garante que vou ter tanto c[1,2,j] quanto c[2,1,j], então não preciso me preocupar com qual vértice escolher pra i e qual pra k na função objetivo

end)
solve(m)

GLPK Integer Optimizer, v4.52
691452 rows, 694080 columns, 2076480 non-zeros
694080 integer variables, all of which are binary
Integer optimization begins...
+348835: mip =     not found yet <=              +inf        (1; 0)
|349500: obj =  1.435995732e+006  infeas = 8.971e-009 (0)
|350000: obj =  1.435995732e+006  infeas = 2.194e-008 (0)
|350121: obj =  1.435995732e+006  infeas = 1.145e-008 (0)
+350121: mip =     not found yet <=  1.435995732e+006        (2; 0)
Time used: 3130.9 secs.  Memory used: 569.6 Mb.
|350500: obj =  1.435995732e+006  infeas = 7.420e-008 (5253)
|351000: obj =  1.435995732e+006  infeas = 2.186e-008 (5253)
|351500: obj =  1.435995732e+006  infeas = 2.717e-008 (5245)
|352000: obj =  1.435995732e+006  infeas = 6.027e-008 (5071)
|352500: obj =  1.435995732e+006  infeas = 9.973e-009 (4799)
|353000: obj =  1.435995732e+006  infeas = 7.327e-009 (4419)
|353500: obj =  1.435995732e+006  infeas = 2.149e-008 (4206)
|354000: obj =  1.435995732e+006  infeas = 3.422e-009 (39

:UserLimit

In [3]:
getobjectivevalue(m)

NaN